# Vaex Run

In [1]:
import os
import vaex as vx
import time
import numpy as np
import shutil
import gc
import joblib
import psutil

In [2]:
# PATH_DATA = '../../df-showdown-data/'
PATH_DATA = 'O:/df-showdown-data/'
TEMP_PATH = PATH_DATA+'temp/'
PATH_OUTPUT = '../output/'

SLA_NAME = 'vaex-sla.pkl'
MEM_NAME = 'vaex-mem.pkl'


In [3]:
all_sla = {}
all_memory = {}

In [4]:
def get_memory():
    est_memory = psutil.virtual_memory()[3]/1000000000
    return est_memory

##     Task1 & Task2- I/O

In [5]:
all_sla['task1'] = {}
all_sla['task1']['csv'] = {}
all_sla['task1']['prq'] = {}

all_sla['task2'] = {}
all_sla['task2']['csv'] = {}
all_sla['task2']['prq'] = {}



all_memory['task1'] = {}
all_memory['task1']['csv'] = {}
all_memory['task1']['prq'] = {}

all_memory['task2'] = {}
all_memory['task2']['csv'] = {}
all_memory['task2']['prq'] = {}


In [6]:
for i in range(10):
    print(i)
    if i<=5:
        fn = 'data_{}.csv'.format(i)
        start = time.perf_counter()
        start_mem = get_memory()
        dtemp1 = vx.open(PATH_DATA+fn)
        end_mem = get_memory() - start_mem
        end = time.perf_counter() - start
        nrows = str(int(dtemp1.shape[0]/1000000))
        all_sla['task1']['csv'][nrows]=end
        all_memory['task1']['csv'][nrows]=end_mem

        start = time.perf_counter()
        start_mem = get_memory()
        dtemp1.export(TEMP_PATH + 'temp.csv')
        end_mem = get_memory() - start_mem
        end = time.perf_counter() - start
        all_sla['task2']['csv'][nrows]=end
        all_memory['task2']['csv'][nrows]=end_mem

        del dtemp1
        time.sleep(3)
        gc.collect()
        os.remove(TEMP_PATH + 'temp.csv')
        time.sleep(2)

    fn = 'data_{}.parquet'.format(i)
    start = time.perf_counter()
    start_mem = get_memory()
    dtemp1 = vx.open(PATH_DATA+fn)
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    nrows = str(int(dtemp1.shape[0]/1000000))
    all_sla['task1']['prq'][nrows]=end
    all_memory['task1']['prq'][nrows]=end_mem
    
    start = time.perf_counter()
    start_mem = get_memory()
    dtemp1.export(TEMP_PATH + 'temp.parquet')
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    all_sla['task2']['prq'][nrows]=end
    all_memory['task2']['prq'][nrows]=end_mem

    del dtemp1
    time.sleep(3)
    gc.collect()
    os.remove(TEMP_PATH + 'temp.parquet')

0
1
2
3
4
5
6
7
8
9


In [8]:
all_sla

{'task1': {'csv': {'0': 1.3412223999999995,
   '1': 0.22003529999999927,
   '5': 0.8697331999999989,
   '10': 1.683620299999987,
   '25': 4.179170499999998,
   '50': 8.248790299999996},
  'prq': {'0': 0.15681350000000194,
   '1': 0.047131499999998994,
   '5': 0.033662699999993606,
   '10': 0.06986320000000035,
   '25': 0.045447200000012344,
   '50': 0.033930499999996755,
   '75': 0.04196869999998398,
   '100': 0.04234879999989971,
   '130': 0.0270077999998648,
   '175': 0.023035300000174175}},
 'task2': {'csv': {'0': 0.9298982999999996,
   '1': 6.534122100000001,
   '5': 18.3984259,
   '10': 32.94440870000001,
   '25': 77.21860610000002,
   '50': 156.6390606},
  'prq': {'0': 0.16862469999999874,
   '1': 1.0122639000000007,
   '5': 5.404940799999991,
   '10': 13.2226529,
   '25': 70.77755460000003,
   '50': 247.51370610000004,
   '75': 622.5552007,
   '100': 1030.5605293999997,
   '130': 1590.1979167999998,
   '175': 2984.5344140999996}}}

In [9]:
all_memory

{'task1': {'csv': {'0': 0.07319142399999912,
   '1': 0.0512860160000006,
   '5': 0.3029729279999991,
   '10': 0.5566914560000011,
   '25': 1.4639431680000001,
   '50': 2.959015936},
  'prq': {'0': 0.012738560000000732,
   '1': -0.0010280960000006445,
   '5': -6.553600000103188e-05,
   '10': 0.0016056319999986357,
   '25': -4.095999999620403e-06,
   '50': -0.00010240000000116822,
   '75': -9.420800000015106e-05,
   '100': 0.0017367039999989231,
   '130': -9.830399999977146e-05,
   '175': 0.0006348799999997823}},
 'task2': {'csv': {'0': 0.054296576000000485,
   '1': 0.15580364799999913,
   '5': -0.0096296959999993,
   '10': -0.565739520000001,
   '25': -1.417261056000001,
   '50': -3.2148807680000004},
  'prq': {'0': 0.015523840000000177,
   '1': 0.029679615999999243,
   '5': 0.25926041600000005,
   '10': 0.35756441600000066,
   '25': 1.2359147519999993,
   '50': 1.0042695679999998,
   '75': 0.3770449920000001,
   '100': 2.22328832,
   '130': -0.3833200639999994,
   '175': 0.174075904000

In [14]:
# del dtemp1
# gc.collect()

## Get Data for other task

In [11]:
all_df = [vx.open(PATH_DATA+'data_{}.parquet'.format(i)) for i in [4,5,6,7]]
df_right = vx.open(PATH_DATA+'data_to_join.parquet')

In [12]:
len(all_df)

4

## Task3 - Sorting

In [13]:
n_task = 'task3'
all_sla[n_task] = {}
all_memory[n_task] = {}

In [14]:
# dtmp = all_df[0]
# dtmp.sort

In [15]:
for dtemp1 in all_df:
    print(round(len(dtemp1)/1000000,2))
    start = time.perf_counter()
    start_mem = get_memory()
    dtemp2 = dtemp1.sort(['Date','Amount'],ascending=[True,False])
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    nrows = str(int(dtemp1.shape[0]/1000000))
    
    all_sla[n_task][nrows]=end
    all_memory[n_task][nrows]=end_mem

    del dtemp1
    del dtemp2
    time.sleep(4)
    gc.collect()
    time.sleep(4)
    gc.collect()


25.02
50.04
75.06
100.08


In [16]:
all_sla[n_task]

{'25': 16.363027899999906,
 '50': 39.31056279999939,
 '75': 71.14714040000035,
 '100': 104.25895479999963}

In [17]:
all_memory[n_task]

{'25': 0.29567795200000013,
 '50': 0.2758205440000001,
 '75': 0.22349823999999963,
 '100': 0.9070714879999997}

## Task4 - Filtering

In [18]:
n_task = 'task4'
all_sla[n_task] = {}
all_memory[n_task] = {}

In [19]:
# dtemp1 = all_df[0]
filt1 = ['Shipped - Delivered to Buyer', 'Cancelled']
for dtemp1 in all_df:
    print(round(len(dtemp1)/1000000,2))
    start = time.perf_counter()
    start_mem = get_memory()
    dtemp2 = dtemp1[(dtemp1['Amount']>300) & (~dtemp1['Status'].isin(filt1))]
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    nrows = str(int(dtemp1.shape[0]/1000000))
    
    all_sla[n_task][nrows]=end
    all_memory[n_task][nrows]=end_mem

    del dtemp1
    del dtemp2
    time.sleep(4)
    gc.collect()
    time.sleep(4)
    gc.collect()
    


25.02
50.04
75.06
100.08


In [20]:
all_sla[n_task]

{'25': 0.7561313000005612,
 '50': 1.187053300000116,
 '75': 1.7472329000001992,
 '100': 2.3464979000000312}

In [21]:
all_memory[n_task]

{'25': 0.04025548800000145,
 '50': 0.06954598400000123,
 '75': 0.08310374400000065,
 '100': 1.0196869119999992}

## Task5 - Merging

In [22]:
n_task = 'task5'
all_sla[n_task] = {}
all_memory[n_task] = {}

In [23]:
for dtemp1 in all_df:
    print(round(len(dtemp1)/1000000,2))
    dtemp1['key'] = dtemp1['Date'] +'_'+dtemp1['ship-service-level']
    df_right['key'] = df_right['Date'] +'_'+df_right['ship-service-level']
    start = time.perf_counter()
    start_mem = get_memory()
    dtemp2 = dtemp1.join(df_right,on='key',how='left',rsuffix='_R')
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    nrows = str(int(dtemp1.shape[0]/1000000))

    all_sla[n_task][nrows]=end
    all_memory[n_task][nrows]=end_mem

    del dtemp1
    del dtemp2
    time.sleep(4)
    gc.collect()
    time.sleep(4)
    gc.collect()

25.02
50.04
75.06
100.08


In [24]:
all_sla[n_task]

{'25': 3.737847000000329,
 '50': 6.0371296000002985,
 '75': 8.571062399999391,
 '100': 11.677208700000847}

In [25]:
all_memory[n_task]

{'25': 0.4650926079999991,
 '50': 0.41904128000000007,
 '75': 0.4814888960000001,
 '100': 0.6101114880000011}

## Task6 - udf apply

In [26]:
n_task = 'task6'
all_sla[n_task] = {}
all_memory[n_task] = {}

In [27]:
def fun(x):
    x0 = x + 1
    for i in range(25):
        if x0<800:
            x0 += i
            x0 = (x0/3.0)*2.5
            x0 = x0*1.2
        else:
            x0 += i/2.0
            x0 = (x0/4.0)*3.8

    return x0

In [28]:
# dtemp1=all_df[0]
# dtemp1['Amount2'] = dtemp1['Amount'].apply(fun)
for dtemp1 in all_df:
    print(round(len(dtemp1)/1000000,2))
    start = time.perf_counter()
    start_mem = get_memory()
    dtemp1['Amount2'] = dtemp1['Amount'].apply(fun)
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    nrows = str(int(dtemp1.shape[0]/1000000))
    
    all_sla[n_task][nrows]=end
    all_memory[n_task][nrows]=end_mem
    
    del dtemp1
    time.sleep(4)
    gc.collect()
    time.sleep(4)
    gc.collect()

25.02
50.04
75.06
100.08


In [29]:
all_sla[n_task]

{'25': 0.018020999999862397,
 '50': 0.010512399999242916,
 '75': 0.01919629999974859,
 '100': 0.012347599999884551}

In [31]:
all_memory[n_task]

{'25': 0.0011673600000001727,
 '50': 0.0,
 '75': -1.2288000000637567e-05,
 '100': 0.0}

## Task7 - aggregation

In [32]:
n_task = 'task7'
all_sla[n_task] = {}
all_memory[n_task] = {}

In [39]:
def p25(x):
    return np.percentile(x,25)

def p75(x):
    return np.percentile(x,75)

In [43]:
# dtemp1=all_df[0]
for dtemp1 in all_df:
    print(round(len(dtemp1)/1000000,2))
    start = time.perf_counter()
    start_mem = get_memory()
    dtemp2 = dtemp1.groupby(['Date','Status']).agg({'Amount':'mean'})
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    nrows = str(int(dtemp1.shape[0]/1000000))
    
    all_sla[n_task][nrows]=end
    all_memory[n_task][nrows]=end_mem

    del dtemp1
    del dtemp2
    time.sleep(4)
    gc.collect()
    time.sleep(4)
    gc.collect()

25.02
50.04
75.06
100.08


In [44]:
all_sla[n_task]

{'25': 11.46278490000077,
 '50': 22.780659199999718,
 '75': 32.29474069999924,
 '100': 43.078597900001114}

In [47]:
all_memory[n_task]

{'25': -1.1242455039999992,
 '50': -0.611667967999999,
 '75': 0.858046464000001,
 '100': 1.3100195840000008}

## Save results

In [48]:
joblib.dump(all_sla,PATH_OUTPUT+SLA_NAME)
joblib.dump(all_memory,PATH_OUTPUT+MEM_NAME)

['../output/vaex-mem.pkl']